# Soil Moisture Import
This file will grab soil moisture data from the DWR field.

In [241]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

Now grab soil moisture data from export file.

In [242]:
df = pd.read_excel('Cherkauer-Export (97).xlsx')
df['Timestamp']=pd.to_datetime(df['Timestamp'])
df['Sensor']=df['Entity Name'].str.split('-').str[-1].astype("int64")

In [243]:
tmpDict={}
for idx in df['Sensor'].unique():
    tmpDF=df[df['Sensor']==idx]
    tmpDF=tmpDF.dropna(how='any')
    middle_index=tmpDF.index[int((len(tmpDF.index)-1)/2+0.5)]
    tmpDict[idx] = tmpDF.loc[middle_index]
df2=pd.DataFrame.from_dict(tmpDict,orient='index')
print(df2)

              Timestamp                                Entity Name  \
115 2023-09-19 00:17:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0115   
557 2023-09-19 00:22:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0557   
558 2023-09-19 00:22:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0558   
629 2023-09-18 23:52:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0629   
630 2023-09-19 00:22:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0630   
111 2023-09-19 00:27:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0111   
112 2023-09-19 00:27:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0112   
113 2023-09-19 00:27:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0113   
114 2023-09-18 23:57:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0114   
116 2023-09-19 00:27:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0116   
118 2023-09-19 00:27:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0118   
554 2023-09-19 00:12:30  ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0554   

     soil_moisture1  soil_moisture2  soil_moisture3  soil_moisture4  \
115            19.

# Field Capacity

Read field capacity

In [244]:
fcdf=pd.read_csv('FC.083023.csv')
fcdf=fcdf.set_index('Sensor')

In [245]:
fcdf=fcdf.loc[df2.index]

In [246]:
fcdf

,fc1,fc2,fc3,fc4
115,23.1,35.8,39.8,47.8
557,27.5,35.3,38.0,39.9
558,36.9,44.1,48.1,46.0
629,36.9,44.9,48.1,46.0
630,26.7,40.3,30.5,25.3
111,31.3,39.9,39.0,36.3
112,37.8,43.4,43.5,44.3
113,33.2,40.4,37.2,32.1
114,37.4,45.0,42.9,43.2
116,45.8,51.4,54.2,57.0


Calculate water deficit by comparing sensor field capacity to sensor moisture

In [247]:
root=24

In [248]:
df2['d1']=0
df2['d2']=0
df2['d3']=0
df2['d4']=0

In [249]:
df2.loc[df2['soil_moisture1'] < fcdf.fc1, 'd1']=fcdf.fc1-df2['soil_moisture1']
df2.loc[df2['soil_moisture2'] < fcdf.fc2, 'd2']=fcdf.fc2-df2['soil_moisture2']
df2.loc[df2['soil_moisture3'] < fcdf.fc3, 'd3']=fcdf.fc3-df2['soil_moisture3']
df2.loc[df2['soil_moisture4'] < fcdf.fc4, 'd4']=fcdf.fc4-df2['soil_moisture4']

In [250]:
df2

,Timestamp,Entity Name,soil_moisture1,soil_moisture2,soil_moisture3,soil_moisture4,soil_temp1,soil_temp2,soil_temp3,soil_temp4,soil_temp5,soil_temp6,soil_temp7,Sensor,d1,d2,d3,d4
115,2023-09-19 00:17:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0115,19.0,24.9,32.8,43.3,14.0,15.0,16.5,16.5,16.0,17.0,17.0,115,4.1,10.9,7.0,4.5
557,2023-09-19 00:22:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0557,22.2,17.8,26.3,29.3,-1567.8,0.0,0.0,0.0,0.0,0.0,0.0,557,5.3,17.5,11.7,10.6
558,2023-09-19 00:22:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0558,33.7,43.4,47.8,45.8,-1567.8,0.0,0.0,0.0,0.0,0.0,0.0,558,3.2,0.7,0.3,0.2
629,2023-09-18 23:52:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0629,7.6,36.4,46.3,46.5,-1567.8,0.0,0.0,0.0,0.0,0.0,0.0,629,29.3,8.5,1.8,0.0
630,2023-09-19 00:22:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0630,19.9,35.5,28.3,25.0,15.5,17.0,16.5,16.5,17.5,18.0,18.0,630,6.8,4.8,2.2,0.3
111,2023-09-19 00:27:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0111,20.9,29.1,29.8,27.6,16.5,17.0,17.0,17.5,17.5,17.5,18.5,111,10.4,10.8,9.2,8.7
112,2023-09-19 00:27:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0112,30.1,32.2,34.0,30.0,16.0,16.0,17.5,16.5,17.0,17.5,17.5,112,7.7,11.2,9.5,14.3
113,2023-09-19 00:27:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0113,27.6,35.7,31.2,28.1,15.0,15.5,16.5,16.0,17.5,16.5,17.0,113,5.6,4.7,6.0,4.0
114,2023-09-18 23:57:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0114,29.9,37.0,33.9,35.9,16.5,16.0,17.0,16.5,17.5,17.0,18.0,114,7.5,8.0,9.0,7.3
116,2023-09-19 00:27:30,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0116,33.7,42.7,49.8,53.3,15.0,16.0,15.5,16.5,16.5,16.5,17.0,116,12.1,8.7,4.4,3.7


In [251]:
if root < 6:
    df2['Deficit'] = (df2['d1'])*root
elif root < 12:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*(root-6)
elif root < 18:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*6 + (df2['d3'])*(root-12)
else:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*6 + (df2['d3'])*6 + (df2['d4'])*(root-18)

In [252]:
df2['Deficit'] = df2['Deficit']/100

In [253]:
df2=df2.sort_index()
df2.transpose()

,111,112,113,114,115,116,118,554,557,558,629,630
Timestamp,2023-09-19 00:27:30,2023-09-19 00:27:30,2023-09-19 00:27:30,2023-09-18 23:57:30,2023-09-19 00:17:30,2023-09-19 00:27:30,2023-09-19 00:27:30,2023-09-19 00:12:30,2023-09-19 00:22:30,2023-09-19 00:22:30,2023-09-18 23:52:30,2023-09-19 00:22:30
Entity Name,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0111,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0112,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0113,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0114,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0115,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0116,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0118,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0554,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0557,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0558,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0629,ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0630
soil_moisture1,20.9,30.1,27.6,29.9,19,33.7,13.4,27.7,22.2,33.7,7.6,19.9
soil_moisture2,29.1,32.2,35.7,37,24.9,42.7,19.2,33.4,17.8,43.4,36.4,35.5
soil_moisture3,29.8,34,31.2,33.9,32.8,49.8,26,37.5,26.3,47.8,46.3,28.3
soil_moisture4,27.6,30,28.1,35.9,43.3,53.3,32.2,44.1,29.3,45.8,46.5,25
soil_temp1,16.5,16,15,16.5,14,15,15,15.5,-1567.8,-1567.8,-1567.8,15.5
soil_temp2,17,16,15.5,16,15,16,16.5,18,0,0,0,17
soil_temp3,17,17.5,16.5,17,16.5,15.5,17,18.5,0,0,0,16.5
soil_temp4,17.5,16.5,16,16.5,16.5,16.5,18.5,19,0,0,0,16.5


In [254]:
df2.to_csv("temp.csv")

Table indexing starts at 0. Gives relationship between sensor id, plot id and treatment.

In [255]:
df3=pd.read_csv('PlotCrosswalk.csv')

In [256]:
df3.head()

,Plot,Sensor,Treatment,Crop
0,103,554,RainStandard,Corn
1,203,112,IrrStandard,Corn
2,206,111,RainIntense,Corn
3,108,114,FertIntense,Corn
4,208,118,FertStandard,Corn


In [257]:
print(df3[df3.Sensor==554].index.values)

[0]
